ASAP CRN Metadata compilation

# Team Hardy. ASAP CRN Metadata scrubbing

confirm with Team Hardy bulkRNAseq metadata

16 Oct 2024
Andy Henrie





In [1]:
import pandas as pd
from pathlib import Path
import os, sys

sys.path.append(os.path.abspath((os.path.join(os.getcwd(), 'src/crn_utils'))))

from util import read_CDE, NULL, prep_table, read_meta_table
from validate import validate_table, ReportCollector
from update_schema import v1_to_v2, v2_to_v3_PMDBS, create_upload_medadata_package

%load_ext autoreload
%autoreload 2


Streamlit NOT successfully imported


## CDEs
load the relavent CDEs

In [2]:
schema_version = "v1"
schema_path = Path.home() / "Projects/ASAP/crn-utils/resource/CDE"
CDEv1 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v2.1"
CDEv2 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v3.0"
CDEv3 = read_CDE(schema_version, local_path=schema_path)

metadata_version: ASAP_CDE_v1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v1.csv
read local file
metadata_version: ASAP_CDE_v2.1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v2.1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v2.1.csv
read local file
metadata_version: ASAP_CDE_v3.0
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v3.0
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v3.0.csv
read local file


> SANITY CHECK: verify reading from google doc works.

```python
CDEv1_ = read_CDE("v1")
CDEv2_ = read_CDE("v2.1")
CDEv3_ = read_CDE("v3.0")
```


## Clean v3 Table


In [3]:

# !gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-hardy/fastq/**/*.gz" > ../hardy_sc_hexhash.log


In [8]:
# bucket_files_md5all

In [5]:
# df = pd.DataFrame(bucket_files_md5.items(), columns=["filename", "md5"])
# df


In [6]:
## convert 
team = "hardy"
collection = "bulk_pmdbs"

root_path = Path.home() / ("Projects/ASAP/data/teams")

metadata_path = root_path / f"{team}/{collection}/metadata/"
v3_path = metadata_path / "v3"
og_path = metadata_path / "og"

v3_meta_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']
og_meta_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_bulkRNAseq']

in_tables = [table_name for table_name in og_meta_tables if f"{table_name}.csv" in os.listdir(og_path)]


### Team Hardy

In [9]:
og_tables = {}
for table_name in in_tables:
    df = read_meta_table(f"{og_path}/{table_name}.csv")
    og_tables[table_name] = df
    

In [10]:
og_tables.keys()

dict_keys(['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'ASSAY_bulkRNAseq'])

In [11]:
og_tables['STUDY']

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,publication_PMID,number_samples,sample_types,types_of_samples,DUA_version,PI_ORCHID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-HARDY,Ryten Lab,Understanding mechanisms of Parkinson's diseas...,Hardy_bulkrnaseq,Hardy_bulkrnaseq_pm_hs,Genetic analysis has identified many risk gene...,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Jonathan Brenton,...,NA,288,"Anterior Cingulate Cortex (ACG), Inferior Pari...",Late stage (Braak 6) PD and control post-morte...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NA,"Version 3.0 - beta, 08/2024",NA,NA


In [12]:
# rationalize the team_dataset_id
og_tables['STUDY']['team_dataset_id'] = collection
# rename PI_ORCHID to PI_ORCID
og_tables['STUDY'].rename(columns={"PI_ORCHID": "PI_ORCID"}, inplace=True)

In [13]:
CDE = CDEv3
dfs = {}
for table,df in og_tables.items():
    if table == "ASSAY_bulkRNAseq":
        table = "ASSAY_RNAseq"

    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()
    dfs[table] = full_table

recoding number_samples as int
🚨⚠️❗ **Missing Required Fields in STUDY: metadata_tables**
🚨⚠️❗ **6 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- metadata_version_date: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- cell_extraction_summary: 1/1 empty rows (REQUIRED)

	- protocols_io_DOI: 1/1 empty rows (REQUIRED)

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
No empty entries (NULL) found .
No invalid entries found in Enum fields.
🚨⚠️❗ **Extra field in SUBJECT: organism**
🚨⚠️❗ **Extra field in SUBJECT: ethnicity**
🚨⚠️❗ **Extra field in

In [14]:
STUDY = dfs['STUDY']
PROTOCOL = dfs['PROTOCOL']
SUBJECT = dfs['SUBJECT']
SAMPLE = dfs['SAMPLE']
DATA = dfs['DATA']
CLINPATH = dfs['CLINPATH']
PMDBS = dfs['PMDBS']
ASSAY_RNAseq = dfs['ASSAY_RNAseq']

In [15]:
STUDY

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,number_samples,sample_types,types_of_samples,DUA_version,PI_ORCID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id,metadata_tables
0,TEAM-HARDY,Ryten Lab,Understanding mechanisms of Parkinson's diseas...,bulk_pmdbs,Hardy_bulkrnaseq_pm_hs,Genetic analysis has identified many risk gene...,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Jonathan Brenton,...,288,"Anterior Cingulate Cortex (ACG), Inferior Pari...",Late stage (Braak 6) PD and control post-morte...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NA,"Version 3.0 - beta, 08/2024",NA,NA,NA


In [16]:
STUDY['team_dataset_id'] = collection
STUDY['metadata_tables'] = f"{v3_meta_tables}"

In [17]:
metadata_version = "v3.0"
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"
STUDY['metadata_version_date'] = METADATA_VERSION_DATE


In [18]:
# fixe the subject_id by adding Hardy and removing the / from the source_subject_id
SUBJECT['subject_id'] = "Hardy_" + SUBJECT['source_subject_id'].str.replace("/", "_")

# the subject_id is actually a sample_id, so we will also need to remove duplicates.
SUBJECT = SUBJECT.drop_duplicates(subset=['subject_id'])



# # fix the 'subject_id' by making a mapper from 'source_subject_id' to 'subject_id' from CLINPATH
# subject_mapper = dict(zip(CLINPATH['source_subject_id'], CLINPATH['subject_id']))
# SUBJECT['subject_id'] = SUBJECT['source_subject_id'].map(subject_mapper)


In [19]:
table = "SUBJECT"
schema = CDE[CDE['Table'] == table]

report = ReportCollector(destination="NA")
full_table, report = validate_table(dfs[table].copy(), table, schema, report)
report.print_log()


All required fields are present in *SUBJECT* table.
No empty entries (NULL) found .
No invalid entries found in Enum fields.
🚨⚠️❗ **Extra field in SUBJECT: organism**
🚨⚠️❗ **Extra field in SUBJECT: ethnicity**
🚨⚠️❗ **Extra field in SUBJECT: GP2_id**
🚨⚠️❗ **Extra field in SUBJECT: age_at_diagnosis**



In [20]:
table = "SAMPLE"
schema = CDE[CDE['Table'] == table]

report = ReportCollector(destination="NA")
full_table, report = validate_table(dfs[table].copy(), table, schema, report)
report.print_log()

recoding replicate_count as int
recoding repeated_sample as int
All required fields are present in *SAMPLE* table.
🚨⚠️❗ **17 Fields with empty (NULL) values:**

	- organism: 288/288 empty rows (REQUIRED)

	- assay_type: 288/288 empty rows (REQUIRED)

	- condition_id: 288/288 empty rows (REQUIRED)

	- organism_ontology_term_id: 288/288 empty rows (REQUIRED)

	- time: 288/288 empty rows (OPTIONAL)

	- development_stage_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- sex_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- self_reported_ethnicity_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- disease_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- tissue_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- assay_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- donor_id: 288/288 empty rows (OPTIONAL)

	- pm_PH: 288/288 empty rows (OPTIONAL)

	- cell_type_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- source_RIN: 288/288 empty rows (OPTIONAL)

	- DV200: 288/288 empty rows

In [21]:
# get organism from SUBJECT
SAMPLE['organism'] = SUBJECT['organism'].iloc[0]
SAMPLE['condition_id'] = SAMPLE['condition'].str.replace(", ", "-").str.replace(" ", "_")
SAMPLE['condition_id'].unique()

# fixe the subject_id by adding Hardy and removing the / from the source_sample_id
SAMPLE['subject_id'] = "Hardy_" + SAMPLE['source_sample_id'].str.replace("/", "_")
SAMPLE['source_sample_id'] = NULL

In [22]:
SUBJECT[['subject_id','source_subject_id','GP2_id','organism','ethnicity','GP2_id','age_at_diagnosis']].head()

,subject_id,source_subject_id,GP2_id,organism,ethnicity,GP2_id,age_at_diagnosis
0,Hardy_P27_11,P27/11,MDGAP-QSBB_000088_s1,Human,NA,MDGAP-QSBB_000088_s1,69
4,Hardy_P8_18,P8/18,MDGAP-QSBB_000583_s1,Human,NA,MDGAP-QSBB_000583_s1,61
8,Hardy_P41_09,P41/09,MDGAP-QSBB_000406_s1,Human,NA,MDGAP-QSBB_000406_s1,NA
12,Hardy_P26_11,P26/11,MDGAP-QSBB_000325_s1,Human,NA,MDGAP-QSBB_000325_s1,NA
16,Hardy_P95_10,P95/10,MDGAP-QSBB_000046_s1,Human,NA,MDGAP-QSBB_000046_s1,71


In [23]:
table = "CLINPATH"
schema = CDE[CDE['Table'] == table]

report = ReportCollector(destination="NA")
full_table, report = validate_table(dfs[table].copy(), table, schema, report)
report.print_log()

recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
All required fields are present in *CLINPATH* table.
🚨⚠️❗ **38 Fields with empty (NULL) values:**

	- duration_pmi: 4/288 empty rows (REQUIRED)

	- AMPPD_id: 288/288 empty rows (OPTIONAL)

	- GP2_id: 288/288 empty rows (OPTIONAL)

	- ethnicity: 288/288 empty rows (OPTIONAL)

	- family_history: 288/288 empty rows (OPTIONAL)

	- last_diagnosis: 288/288 empty rows (OPTIONAL)

	- age_at_onset: 288/288 empty rows (OPTIONAL)

	- age_at_diagnosis: 288/288 empty rows (OPTIONAL)

	- first_motor_symptom: 288/288 empty rows (OPTIONAL)

	- hx_dementia_mci: 288/288 empty rows (OPTIONAL)

	- hx_melanoma: 288/288 empty rows (OPTIONAL)

	- education_level: 288/288 empty rows (OPTIONAL)

	- smoking_status: 288/288 empty rows (OPTIONAL)

	- smoking_years: 288/288 empty rows (OPTIONAL)

	- APOE_e4_status: 288/288 empty rows (OPTIONAL)

	- cognitive_status: 288/288 empty rows (OPTIONAL)

	- time_from_baseline

In [24]:
# copy 'GP2_id', 'age_at_diagnosis' from SUBJECT
gp2_mapper = dict(zip(SUBJECT['subject_id'], SUBJECT['GP2_id']))
age_mapper = dict(zip(SUBJECT['subject_id'], SUBJECT['age_at_diagnosis']))


# fixe the subject_id by adding Hardy and removing the / from the source_subject_id
CLINPATH['subject_id'] = "Hardy_" + CLINPATH['source_subject_id'].str.replace("/", "_")

# the subject_id is actually a sample_id, so we will also need to remove duplicates.
CLINPATH = CLINPATH.drop_duplicates(subset=['subject_id'])

CLINPATH['GP2_id'] = CLINPATH['subject_id'].map(gp2_mapper)
CLINPATH['age_at_diagnosis'] = CLINPATH['subject_id'].map(age_mapper)
CLINPATH[['subject_id','source_subject_id','GP2_id','age_at_diagnosis']]

/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_85600/1833959381.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CLINPATH['GP2_id'] = CLINPATH['subject_id'].map(gp2_mapper)
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_85600/1833959381.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CLINPATH['age_at_diagnosis'] = CLINPATH['subject_id'].map(age_mapper)


,subject_id,source_subject_id,GP2_id,age_at_diagnosis
0,Hardy_P27_11,P27/11,MDGAP-QSBB_000088_s1,69
4,Hardy_P8_18,P8/18,MDGAP-QSBB_000583_s1,61
8,Hardy_P41_09,P41/09,MDGAP-QSBB_000406_s1,NA
12,Hardy_P26_11,P26/11,MDGAP-QSBB_000325_s1,NA
16,Hardy_P95_10,P95/10,MDGAP-QSBB_000046_s1,71
...,...,...,...,...
268,Hardy_P11_17,P11/17,MDGAP-DEMENTIA_000251_s1,NA
272,Hardy_P62_15,P62/15,MDGAP-QSBB_000517_s1,80
276,Hardy_P68_05,P68/05,MDGAP-QSBB_000014_s1,62
280,Hardy_P52_13,P52/13,MDGAP-QSBB_000081_s1,64


In [25]:
table = "SAMPLE"
schema = CDE[CDE['Table'] == table]

report = ReportCollector(destination="NA")
full_table, report = validate_table(dfs[table].copy(), table, schema, report)
report.print_log()

recoding replicate_count as int
recoding repeated_sample as int
All required fields are present in *SAMPLE* table.
🚨⚠️❗ **16 Fields with empty (NULL) values:**

	- source_sample_id: 288/288 empty rows (REQUIRED)

	- assay_type: 288/288 empty rows (REQUIRED)

	- organism_ontology_term_id: 288/288 empty rows (REQUIRED)

	- time: 288/288 empty rows (OPTIONAL)

	- development_stage_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- sex_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- self_reported_ethnicity_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- disease_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- tissue_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- assay_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- donor_id: 288/288 empty rows (OPTIONAL)

	- pm_PH: 288/288 empty rows (OPTIONAL)

	- cell_type_ontology_term_id: 288/288 empty rows (OPTIONAL)

	- source_RIN: 288/288 empty rows (OPTIONAL)

	- DV200: 288/288 empty rows (OPTIONAL)

	- suspension_type: 288/288

In [26]:
from update_schema import intervention_typer
# CONDITION
# construct this table.  needs to be checked by hand
CONDITIONv3 = pd.DataFrame(columns=CDEv3[CDEv3['Table'] == "CONDITION"]['Field'])
CONDITIONv3['condition_id'] = SUBJECT['primary_diagnosis'].unique()
CONDITIONv3['intervention_name'] = "Case-Control"
CONDITIONv3['intervention_id'] = CONDITIONv3['condition_id'].apply(intervention_typer)
CONDITIONv3['condition_id'] = CONDITIONv3['condition_id'].str.lower().str.replace(" ", "_")

CONDITIONv3

Field,condition_id,intervention_name,intervention_id,protocol_id,intervention_aux_table
0,idiopathic_pd,Case-Control,Case,NaN,NaN
1,no_pd_nor_other_neurological_disorder,Case-Control,Control,NaN,NaN
2,other_neurological_disorder,Case-Control,Other,NaN,NaN


In [27]:
# now save the tables
v3_tables = {
    "STUDY": STUDY,
    "PROTOCOL": PROTOCOL,
    "SUBJECT": SUBJECT,
    "SAMPLE": SAMPLE,
    "DATA": DATA,
    "CLINPATH": CLINPATH,
    "PMDBS": PMDBS,
    "CONDITION": CONDITIONv3,
    "ASSAY_RNAseq": ASSAY_RNAseq
}

In [28]:

for table,df in v3_tables.items():
    schema = CDE[CDE['Table'] == table]
    valid_fields = schema['Field'].unique()
    df_out = df[valid_fields]
    aux_fields = set(df.columns) - set(valid_fields)
    if aux_fields:
        df_aux = df[list(aux_fields)]
        df_aux.to_csv(og_path / f"{table}_auxiliary.csv", index=False)
        print(f"Saved {table}_auxiliary.csv")
    df_out.to_csv(v3_path / f"{table}.csv", index=False)

Saved SUBJECT_auxiliary.csv
Saved SAMPLE_auxiliary.csv
Saved ASSAY_RNAseq_auxiliary.csv


In [29]:
CDE = CDEv3
tables = CDE['Table'].unique()
v3_meta_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']

v3_tables = {}
for table in v3_meta_tables:
    df = read_meta_table(v3_path / f"{table}.csv")
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()
    v3_tables[table] = full_table

recoding number_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **5 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- cell_extraction_summary: 1/1 empty rows (REQUIRED)

	- protocols_io_DOI: 1/1 empty rows (REQUIRED)

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
No empty entries (NULL) found .
No invalid entries found in Enum fields.

recoding replicate_count as int
recoding repeated_sample as int
All required fields are present in *SAMPLE* table.
🚨⚠️❗ **16 Fields with empty (NULL) values:**

	- sou

In [30]:
### Create metadata package

export_path = root_path / f"{team}"

create_upload_medadata_package(export_path, v3_tables)
v3_tables['SAMPLE']['condition_id']
v3_tables['CONDITION']

,condition_id,intervention_name,intervention_id,protocol_id,intervention_aux_table
0,idiopathic_pd,Case-Control,Case,NA,NA
1,no_pd_nor_other_neurological_disorder,Case-Control,Control,NA,NA
2,other_neurological_disorder,Case-Control,Other,NA,NA


## import uploaded Table
write clean metadata tables according to CDE v1

In [ ]:
## convert 
metadata_path = Path.home() / ("Projects/ASAP/meta-clean") / "clean/team-Lee/v2"

# SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.csv", index_col=0)
SUBJECT = read_file(f"{metadata_path}/SUBJECT.csv")
# SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv", index_col=0)
SAMPLE = read_file(f"{metadata_path}/SAMPLE.csv")
# CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv", index_col=0)
CLINPATH = read_file(f"{metadata_path}/CLINPATH.csv")

# STUDY = pd.read_csv(f"{metadata_path}/STUDY.csv", index_col=0)
STUDY = read_file(f"{metadata_path}/STUDY.csv")
# PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.csv", index_col=0)
PROTOCOL = read_file(f"{metadata_path}/PROTOCOL.csv")
# DATA = pd.read_csv(f"{metadata_path}/DATA.csv", index_col=0)
DATA = read_file(f"{metadata_path}/DATA.csv")


reading /Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Lee/v2/SUBJECT.csv txt/csv, encoding=latin1
reading /Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Lee/v2/SAMPLE.csv txt/csv, encoding=latin1
reading /Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Lee/v2/CLINPATH.csv txt/csv, encoding=latin1
reading /Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Lee/v2/STUDY.csv txt/csv, encoding=latin1
reading /Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Lee/v2/PROTOCOL.csv txt/csv, encoding=latin1
reading /Users/ergonyc/Projects/ASAP/meta-clean/clean/team-Lee/v2/DATA.csv txt/csv, encoding=latin1


In [ ]:
bool(Path.cwd())

True

In [ ]:
metadata_version = "v3.0-beta"
CDE_df = read_CDE(metadata_version, local_path=Path.cwd())




metadata_version: ASAP_CDE_v3.0.0-beta
/Users/ergonyc/Projects/ASAP/harmonized-wf-dev/data/teams/lee/ASAP_CDE_v3.0.0-beta.csv
read local file


In [ ]:
CDE_df.head()

,Table,Field,Description,DataType,Required,Validation,V0,comment,denormalized,dataset relavent
2,STUDY,ASAP_team_name,ASAP Team Name: Name of the ASAP CRN Team. i...,Enum,Required,"[""TEAM-LEE"",""TEAM-HAFLER"",""TEAM-HARDY"", ""TEAM-...",NaN,NaN,NaN,NaN
3,STUDY,ASAP_lab_name,Lab Name. : Lab name that is submitting data...,String,Required,NaN,NaN,NaN,NaN,NaN
4,STUDY,project_name,Project Name: A Title of the overall project...,String,Required,NaN,NaN,NaN,NaN,NaN
5,STUDY,team_dataset_id,"The ""project_name"" is often too verbose for pr...",String,Required,NaN,NaN,NaN,NaN,NaN
6,STUDY,project_dataset,Dataset Name: A unique name is required for ...,String,Required,NaN,NaN,NaN,NaN,NaN


In [ ]:
DATA.head()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name,file_description,file_MD5,technology,omic,adjustment,content,time,header,annotation,configuration_file
0,MFG_HC_1225,rep1,1,0,BATCH_4,fastq,MFGHC1225_S9_L001_R1_001.fastq.gz,Raw sequencing data,9977258e598d6a52130c29c71aef6925,SN,RNA,Raw,Reads,0,NA,NA,NA(raw data)
1,MFG_HC_1225,rep1,1,0,BATCH_4,fastq,MFGHC1225_S9_L001_R2_001.fastq.gz,Raw sequencing data,fe2cf93257801227b7072a4fb7d18792,SN,RNA,Raw,Reads,0,NA,NA,NA(raw data)
2,MFG_HC_0602,rep1,1,0,BATCH_4,fastq,MFGHC0602_S2_L001_R1_001.fastq.gz,Raw sequencing data,110ca4864cf6938faca67567bebfb6cc,SN,RNA,Raw,Reads,0,NA,NA,NA(raw data)
3,MFG_HC_0602,rep1,1,0,BATCH_4,fastq,MFGHC0602_S2_L001_R2_001.fastq.gz,Raw sequencing data,0dcc67217e43ab53bae0d0676f9bfe8b,SN,RNA,Raw,Reads,0,NA,NA,NA(raw data)
4,MFG_PD_0009,rep1,1,0,BATCH_4,fastq,MFGPD0009_S3_L001_R1_001.fastq.gz,Raw sequencing data,a2608d0bd192333b0076d7091c1c50ea,SN,RNA,Raw,Reads,0,NA,NA,NA(raw data)


In [ ]:
SAMPLE.head()

,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,MFG_HC_1225,HC_1225,12-25,rep1,1,0,BATCH_4,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
1,HIP_HC_1225,HC_1225,12-25,rep1,1,0,BATCH_9,Brain,Hippocampus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
2,SN_HC_1225,HC_1225,12-25,rep1,1,0,BATCH_7,Brain,Substantia_Nigra,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
3,MFG_HC_0602,HC_0602,06-02,rep1,1,0,BATCH_4,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
4,HIP_HC_0602,HC_0602,06-02,rep1,1,0,BATCH_9,Brain,Hippocampus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA


In [ ]:
# merge df and DATA on sample_id
DATA_ = df.merge(DATA, left_on="sample_id", right_on="sample_id", how="left")


In [ ]:
# drop duplicates on md5
DATA_.drop_duplicates(subset="md5", inplace=True)

df.shape, DATA.shape, DATA_.shape

((50, 5), (150, 17), (50, 21))

In [ ]:
DATA_.columns

Index(['filename', 'md5', 'id', 'subject_id', 'sample_id', 'replicate',
       'replicate_count', 'repeated_sample', 'batch', 'file_type', 'file_name',
       'file_description', 'file_MD5', 'technology', 'omic', 'adjustment',
       'content', 'time', 'header', 'annotation', 'configuration_file'],
      dtype='object')

copy 'filename' -> 'file_name'

copy  'md5' -> 'file_md5'

change 'technology' -> 'bulk'

In [ ]:
DATA_['file_MD5'] = DATA_['md5']
DATA_['file_name'] = DATA_['filename']
DATA_['technology'] = 'bulk'

# set batch to "batch_x"
DATA_['batch'] = "batch_x"

Next take the unique sample_id and subset SAMPLE, and CLINPATH to only include the bulk samples.


In [ ]:
sample_ids = DATA_['sample_id'].unique()
subject_ids = DATA_['subject_id'].unique()
sample_ids, subject_ids

(array(['MFG_PD_0009', 'MFG_PD_0348', 'MFG_PD_0413', 'MFG_HC_0602',
        'MFG_HC_1225', 'MFG_HC_1308', 'MFG_PD_1312', 'MFG_PD_1317',
        'MFG_PD_1344', 'MFG_PD_1441', 'MFG_PD_1504', 'MFG_PD_1858',
        'MFG_HC_1862', 'MFG_HC_1864', 'MFG_PD_1902', 'MFG_PD_1921',
        'MFG_HC_1939', 'MFG_PD_1973', 'MFG_PD_2005', 'MFG_PD_2038',
        'MFG_HC_2057', 'MFG_PD_2058', 'MFG_HC_2061', 'MFG_HC_2062',
        'MFG_HC_2067'], dtype=object),
 array(['PD_0009', 'PD_0348', 'PD_0413', 'HC_0602', 'HC_1225', 'HC_1308',
        'PD_1312', 'PD_1317', 'PD_1344', 'PD_1441', 'PD_1504', 'PD_1858',
        'HC_1862', 'HC_1864', 'PD_1902', 'PD_1921', 'HC_1939', 'PD_1973',
        'PD_2005', 'PD_2038', 'HC_2057', 'PD_2058', 'HC_2061', 'HC_2062',
        'HC_2067'], dtype=object))

In [ ]:
SAMPLE_ = SAMPLE[SAMPLE['sample_id'].isin(sample_ids)].copy()
CLINPATH_ = CLINPATH[CLINPATH['subject_id'].isin(subject_ids)].copy()


In [ ]:

SUBJECT_ = SUBJECT[SUBJECT['subject_id'].isin(subject_ids)].copy()

In [ ]:
# reindex SAMPLE_ and CLINPATH_
SAMPLE_.reset_index(inplace=True, drop=True)
CLINPATH_.reset_index(inplace=True, drop=True)


# set batch to batch_x
SAMPLE_['batch'] = "batch_x"

In [ ]:
SAMPLE_

,sample_id,subject_id,source_sample_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,hemisphere,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV200,pm_PH,donor_id
0,MFG_HC_1225,HC_1225,12-25,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
1,MFG_HC_0602,HC_0602,06-02,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
2,MFG_PD_0009,PD_0009,00-09,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
3,MFG_PD_1921,PD_1921,19-21,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
4,MFG_PD_2058,PD_2058,20-58,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
5,MFG_PD_1441,PD_1441,14-41,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000383 (female),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
6,MFG_PD_1344,PD_1344,13-44,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000383 (female),Unknown,MONDO:0005180,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
7,MFG_HC_1939,HC_1939,19-39,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000383 (female),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
8,MFG_HC_1308,HC_1308,13-08,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA
9,MFG_HC_1862,HC_1862,18-62,rep1,1,0,batch_x,Brain,Middle_Frontal_Gyrus,Unknown,...,PATO:0000384 (male),Unknown,PATO:0000461,UBERON:0002702,NA(multiple),EFO:0030004,nucleus,NA,NA,NA


In [ ]:
report = ReportCollector(destination="log")

df = SAMPLE_.copy()
table_choice = "SAMPLE"
# perform the valadation

# specific_cde_df = CDE_df[CDE_df['Table'] == table_choice]
specific_cde_df = CDE_df[CDE_df['Table'].str.startswith(table_choice)]

retval = validate_table(df, table_choice, specific_cde_df, report)

report.print_log()

validating sample_id
validating subject_id
validating source_sample_id
validating replicate
validating replicate_count
validating repeated_sample
validating batch
validating condition
validating tissue
validating time
validating alternate_id
All required fields are present in *SAMPLE* table.
🚨⚠️❗ **Missing Optional Fields in SAMPLE: condition, time, alternate_id**
No empty entries (NULL) found .
No invalid entries found in Enum fields.



In [ ]:
SAMPLE_export, SAMPLE_out, report = retval

In [ ]:
SAMPLE_export.shape, SAMPLE_.shape, SAMPLE.shape, SAMPLE_out.shape

((25, 33), (25, 33), (75, 33), (25, 11))

In [ ]:
report = ReportCollector(destination="log")

df = CLINPATH_.copy()
table_choice = "CLINPATH"
# perform the valadation

# specific_cde_df = CDE_df[CDE_df['Table'] == table_choice]
specific_cde_df = CDE_df[CDE_df['Table'].str.startswith(table_choice)]

retval = validate_table(df, table_choice, specific_cde_df, report)

report.print_log()

CLINPATH_export, CLINPATH_out, report = retval



validating subject_id
validating source_subject_id
validating duration_pmi
validating age_at_death
validating path_autopsy_dx_main
validating path_autopsy_second_dx
validating path_autopsy_third_dx
validating path_autopsy_fourth_dx
validating path_autopsy_fifth_dx
validating path_autopsy_sixth_dx
validating path_autopsy_seventh_dx
validating path_autopsy_eight_dx
validating path_year_death
validating cause_death
validating other_cause_death_1
validating other_cause_death_2
validating brain_weight
validating path_braak_nft
validating path_braak_asyn
validating path_cerad
validating path_thal
validating known_pathogenic_mutation
validating PD_pathogenic_mutation
validating path_mckeith
validating sn_neuronal_loss
validating path_infarcs
validating path_nia_ri
validating path_nia_aa_a
validating path_nia_aa_b
validating path_nia_aa_c
validating TDP43
validating arteriolosclerosis_severity_scale
validating amyloid_angiopathy_severity_scale
validating path_ad_level
validating dig_slide_avai

In [ ]:
import numpy as np
df.replace({"":NULL, pd.NA:NULL, np.NaN:NULL}, inplace=True)


## Clean V1 Table
write clean metadata tables according to CDE v1

### Team Hardy

In [3]:
## convert 
data_path = Path.home() / ("Projects/ASAP/team-hardy")
metadata_path = data_path / "metadata"

SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.csv")
CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv")
STUDY = pd.read_csv(f"{metadata_path}/STUDY.csv")
PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.csv")
SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv")

metadata_version = "v1"

CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"

read url


In [4]:

# fix STUDY formatting
tmp = pd.DataFrame()
tmp = STUDY[["name","value"]].transpose().reset_index().drop(columns=["index"])
tmp

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,aine.fairbrother-browne.18@ucl.ac.uk,...,NaN,NaN,NaN,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NaN,"Version 1, 09/2023"


In [5]:

tmp.columns = tmp.iloc[0]
STUDY = tmp.drop([0])
STUDY.head()


,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,aine.fairbrother-browne.18@ucl.ac.uk,...,NaN,NaN,NaN,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NaN,"Version 1, 09/2023"


In [6]:

STUDY = prep_table(STUDY, CDE_df)
# update metadata_version_date
STUDY['metadata_version_date'] = METADATA_VERSION_DATE


In [7]:
report = ReportCollector(destination="")
validate_table(STUDY, "STUDY", CDE_df, report)
report.print_log()


All required fields are present in *STUDY* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in STUDY. 🥳



In [8]:
STUDY.head()

,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,Mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,Aine.fairbrother-browne.18@ucl.ac.uk,...,Nan,Nan,Nan,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,Https://scholar.google.co.uk/citations?user=lt...,Nan,v1_20240516


In [9]:
# fix STUDY formatting
tmp = pd.DataFrame()
tmp = PROTOCOL[["name","value"]].transpose().reset_index().drop(columns=["index"])
tmp.columns = tmp.iloc[0]
PROTOCOL = tmp.drop([0])
PROTOCOL.head()

,sample_collection_summary,cell_extraction_summary,lib_prep_summary,data_processing_summary,github_url,protocols_io_DOI,other_reference
1,"This dataset contains cortical regions only, p...",From protocols.io: This protocol is used to is...,'Nuclei were extracted from homogenised post-m...,Cell ranger was used to convert raw sequencing...,Raw to fastq to mapped: https://github.com/RHR...,Nuclear extraction protocol: 10.17504/protocol...,NaN


In [10]:
# fix the column order
PROTOCOL = prep_table(PROTOCOL, CDE_df)
report = ReportCollector(destination="")
validate_table(PROTOCOL, "PROTOCOL", CDE_df, report)
report.print_log()



All required fields are present in *PROTOCOL* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in PROTOCOL. 🥳



In [11]:

SUBJECT = prep_table(SUBJECT, CDE_df)

report = ReportCollector(destination="")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()


All required fields are present in *SUBJECT* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- duration_pmi: 1/64 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in SUBJECT. 🥳



In [12]:
SUBJECT.race.unique()[0]

'Nan'

In [13]:
SUBJECT = reorder_table_to_CDE(SUBJECT, "SUBJECT", CDE_df)
SUBJECT.head()

,subject_id,source_subject_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,duration_pmi,primary_diagnosis,primary_diagnosis_text
0,Babom,P2/14,QSBB_UK,Human,Female,78,Nan,Nan,46.0,Idiopathic PD,clinpath info: PDD | PD (with dementia)
1,Borah,P4/11,QSBB_UK,Human,Male,63,Nan,Nan,37.0,Idiopathic PD,clinpath info: PD | PD
2,Bovon,P95/10,QSBB_UK,Human,Male,81,Nan,Nan,59.5,Idiopathic PD,clinpath info: PD | NA
3,Davof,P80/11,QSBB_UK,Human,Male,80,Nan,Nan,100.0,Idiopathic PD,clinpath info: PD | PD
4,Dudug,P82/10,QSBB_UK,Human,Female,87,Nan,Nan,84.0,No PD nor other neurological disorder,clinpath info: Control | Control


In [14]:
# def clean_empty_cols(df):
#     for col in df.columns:
#         if df[col].dtype in ["object", "string"]:
#             df[col].replace("Nan", pd.NA, inplace=True)
#             df[col].replace("nan", pd.NA, inplace=True)
#             df[col].replace("NA", pd.NA, inplace=True)

#         elif df[col].dtype in ["int", "float"]:
#             df[col].fillna(pd.NA, inplace=True)
#     return df


# SUBJECT = clean_empty_cols(SUBJECT)

SUBJECT.replace("Nan", pd.NA, inplace=True)
# SUBJECT.replace("nan", "", inplace=True)
# SUBJECT.fillna("", inplace=True)
SUBJECT.head()

,subject_id,source_subject_id,biobank_name,organism,sex,age_at_collection,race,ethnicity,duration_pmi,primary_diagnosis,primary_diagnosis_text
0,Babom,P2/14,QSBB_UK,Human,Female,78,<NA>,<NA>,46.0,Idiopathic PD,clinpath info: PDD | PD (with dementia)
1,Borah,P4/11,QSBB_UK,Human,Male,63,<NA>,<NA>,37.0,Idiopathic PD,clinpath info: PD | PD
2,Bovon,P95/10,QSBB_UK,Human,Male,81,<NA>,<NA>,59.5,Idiopathic PD,clinpath info: PD | NA
3,Davof,P80/11,QSBB_UK,Human,Male,80,<NA>,<NA>,100.0,Idiopathic PD,clinpath info: PD | PD
4,Dudug,P82/10,QSBB_UK,Human,Female,87,<NA>,<NA>,84.0,No PD nor other neurological disorder,clinpath info: Control | Control


In [15]:
SAMPLE[['assay','sequencing_end', 'sequencing_length', 'sequencing_instrument']]

,assay,sequencing_end,sequencing_length,sequencing_instrument
0,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
1,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
2,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
4,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
...,...,...,...,...
3611,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3612,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3613,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3614,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000


In [47]:
SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv")

# force the right sex_ontology_term_id
SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"

SAMPLE = prep_table(SAMPLE, CDE_df)

# allow sequence_length == 190 for now

# SAMPLE.fillna("", inplace=True)
# remove any text nans
SAMPLE.replace("Nan", pd.NA, inplace=True)
SAMPLE.replace("nan", pd.NA, inplace=True)
SAMPLE.replace("NA", pd.NA, inplace=True)
# SAMPLE = clean_empty_cols(SUBJECT)
# SAMPLE.fillna("", inplace=True)

report = ReportCollector(destination="")
validate_table(SAMPLE, "SAMPLE", CDE_df, report)
report.print_log()




# add 'replicate' coding (nans)

🚨⚠️❗ **Missing Required Fields in SAMPLE: source_sample_id**
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- source_RIN: 3616/3616 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- pm_PH: 3616/3616 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*sequencing_length*_:  invalid values 💩'190'
    - valid ➡️ '25', '50', '100', '150'



In [48]:

SAMPLE['source_sample_id'] = pd.NA
SAMPLE['DV200'] = pd.NA
# SAMPLE = reorder_table_to_CDE(SAMPLE, "SAMPLE", CDE_df)


In [49]:
SAMPLE['replicate'].replace({pd.NA: "Rep1", "rep1": "Rep1", "rep2": "Rep2"}, inplace=True)

In [50]:
SAMPLE.head()

,sample_id,source_subject_id,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,source_RIN,...,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV2000,pm_PH,donor_id,source_sample_id,DV200
0,Babom_ACG,P2/14,Babom,Rep1,1,0,2,Brain,ACG,NaN,...,MONDO:0005180,UBERON:0009835,<NA>,EFO:0008913,nucleus,NaN,NaN,<NA>,<NA>,<NA>
1,Babom_ACG,P2/14,Babom,Rep1,1,0,2,Brain,ACG,NaN,...,MONDO:0005180,UBERON:0009835,<NA>,EFO:0008913,nucleus,NaN,NaN,<NA>,<NA>,<NA>
2,Babom_ACG,P2/14,Babom,Rep1,1,0,2,Brain,ACG,NaN,...,MONDO:0005180,UBERON:0009835,<NA>,EFO:0008913,nucleus,NaN,NaN,<NA>,<NA>,<NA>
3,Babom_ACG,P2/14,Babom,Rep1,1,0,2,Brain,ACG,NaN,...,MONDO:0005180,UBERON:0009835,<NA>,EFO:0008913,nucleus,NaN,NaN,<NA>,<NA>,<NA>
4,Babom_ACG,P2/14,Babom,Rep1,1,0,2,Brain,ACG,NaN,...,MONDO:0005180,UBERON:0009835,<NA>,EFO:0008913,nucleus,NaN,NaN,<NA>,<NA>,<NA>


In [51]:
CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv")

CLINPATH = prep_table(CLINPATH, CDE_df)


# CLINPATH.replace("Nan", pd.NA, inplace=True)
CLINPATH.replace("nan", "Nan", inplace=True)
CLINPATH.replace("NA", "Nan", inplace=True)

# CLINPATH.fillna("Nan")
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()




🚨⚠️❗ **Missing Required Fields in CLINPATH: source_sample_id**
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 138/138 empty rows

	- age_at_diagnosis: 10/138 empty rows

	- first_motor_symptom: 138/138 empty rows

	- path_year_death: 138/138 empty rows

	- brain_weight: 138/138 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 138/138 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*path_autopsy_dx_main*_:  invalid values 💩'Control brain', 'Pathological ageing', 'Control brain / Path ageing', 'Argyrophilic grain disease', 'Control brain, Cerebrovascular disease (small vessel)', 'Cerebrovascular disease (small vessel)', 'Control brain, Alzheimer`s disease (intermediate level AD pathological change)', 'Control brain / Path ageing, CAA'
    - valid ➡️ 'Lewy body disease nos', 'Parkinson's disease', 'Parkinson's disease with dementia', 'Dementia with Lewy bodies', 'Multiple system atrophy (SND>OPCA)', 'Multiple system atrophy (OPCA<SND)', 

In [52]:
na_float2intstr = lambda val: str(int(float(val))) if val != 'Nan' else pd.NA 

# replace 'path_braak_asyn' with with string of the numeric. converte nan to ""
CLINPATH['path_braak_asyn'] = CLINPATH['path_braak_asyn'].apply(na_float2intstr)



In [53]:

# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_braak_nft'] = CLINPATH['path_braak_nft'].apply(na_float2intstr).replace({"0":"0", 
                                                                                        "1":"I", 
                                                                                        "2": "II", 
                                                                                        "3":"III", 
                                                                                        "4":"IV", 
                                                                                        "5":"V", 
                                                                                        "6":"VI"})


In [54]:

# code family_history as "Not Reported" (currently empty)
CLINPATH['family_history'] = "Not Reported"



# check APOE_e4_status ? currently empty

# `path_autopsy_dx_main`  actually seems good parser might be wrong

# code "at least 4" as "4/5" 

CLINPATH['path_thal'] = CLINPATH['path_thal'].replace({'At least 4':"4/5"})


CLINPATH['path_mckeith'] = CLINPATH['path_mckeith'].replace({'Diffuse neocortical': "Diffuse, neocortical (brainstem, limbic and neocortical involvement)", 
                                                        'Limbic transitional': "Limbic (transitional)" ,
                                                        'Diffuse Neocortical':"Diffuse, neocortical (brainstem, limbic and neocortical involvement)"})


In [55]:

CLINPATH['path_nia_aa_a'] = CLINPATH['path_nia_aa_a'].apply(na_float2intstr).replace({"0":"A0", 
                                                                                    "1":"A1", 
                                                                                    "2": "A2", 
                                                                                    "3":"A3"})


In [56]:

# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_nia_aa_b'] = CLINPATH['path_nia_aa_b'].apply(na_float2intstr).replace({"0":"B0", 
                                                                                    "1":"B1", 
                                                                                    "2": "B2", 
                                                                                    "3":"B3"})


# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_nia_aa_c'] = CLINPATH['path_nia_aa_c'].apply(na_float2intstr).replace({"0":"C0", 
                                                                                    "1":"C1", 
                                                                                    "2": "C2", 
                                                                                    "3":"C3"})



In [57]:



CLINPATH['path_ad_level'] = CLINPATH['path_ad_level'].replace({"No evidence": "No evidence of Alzheimer\'s disease neuropathological change"})





In [58]:
path_autopsy_map = { "Parkinson's disease with dementia": "Parkinson's disease with dementia", 
       "Parkinson's disease": "Parkinson's disease",
       'Control brain':"Control, no misfolded protein or significant vascular pathology", 
       'Pathological ageing': 'Control, no misfolded protein or significant vascular pathology',
       'Control brain / Path ageing': 'Control, no misfolded protein or significant vascular pathology',
       'Argyrophilic grain disease': "Control, Argyrophilic grain disease",
       'Control brain, Cerebrovascular disease (small vessel)':"Control, Cerebrovascular disease (atherosclerosis)",
       'Cerebrovascular disease (small vessel)':"Control, Cerebrovascular disease (atherosclerosis)",
       "Control brain, Alzheimer`s disease (intermediate level AD pathological change)":"Alzheimer's disease (intermediate level neuropathological change)",
       'Control brain / Path ageing, CAA':"Control, Cerebrovascular disease (cerebral amyloid angiopathy)"}


In [59]:
CLINPATH['path_autopsy_dx_main'] = CLINPATH['path_autopsy_dx_main'].replace(path_autopsy_map)

In [60]:
CLINPATH['path_autopsy_dx_main'].unique()

array(["Parkinson's disease with dementia", "Parkinson's disease",
       'Control, no misfolded protein or significant vascular pathology',
       'Control, Argyrophilic grain disease',
       'Control, Cerebrovascular disease (atherosclerosis)',
       "Alzheimer's disease (intermediate level neuropathological change)",
       'Control, Cerebrovascular disease (cerebral amyloid angiopathy)',
       'Nan'], dtype=object)

In [61]:
# remove any text nans
CLINPATH.replace("Nan", pd.NA, inplace=True)



In [62]:
CLINPATH['source_sample_id'] = pd.NA

# CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE_df)


In [63]:

report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()




All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- age_at_onset: 138/138 empty rows

	- age_at_diagnosis: 10/138 empty rows

	- first_motor_symptom: 138/138 empty rows

	- path_year_death: 138/138 empty rows

	- brain_weight: 138/138 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 138/138 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*hx_dementia_mci*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No'
- _*hx_melanoma*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No'
- _*education_level*_:  invalid values 💩'<NA>'
    - valid ➡️ 'High School', 'High School/GED', 'Some college without degree', 'Associate degree college', 'Bachelor's degree', 'Master's degree', 'Professional or doctoral degree', 'Refuse', 'Other'
- _*smoking_status*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Current smoker', 'Former smoker', 'Never', 'Unknown'
- _*APOE_e4_status*_:  invalid values 💩'<NA>'
    - valid ➡️ '22', '23

In [64]:


# # write the clean metadata
# STUDY.to_csv(data_path / "metadata/STUDY.csv")
# PROTOCOL.to_csv(data_path / "metadata/PROTOCOL.csv")
# CLINPATH.to_csv(data_path / "metadata/CLINPATH.csv")
# SAMPLE.to_csv(data_path / "metadata/SAMPLE.csv")
# SUBJECT.to_csv(data_path / "metadata/SUBJECT.csv")

# # also writh them to clean...
# 
#  

export_root = Path.cwd() / "clean/team-Hardy"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")



## Update the table to v2

In [65]:
from update_schema import update_tables_to_CDEv2

tables_path = Path.cwd() / "clean/team-Hardy"


CDEv1 = pd.read_csv( Path.cwd() / "ASAP_CDE_v1.csv" )
CDEv2 = pd.read_csv( Path.cwd() / "ASAP_CDE_v2.csv" )


STUDYv2, PROTOCOLv2, SAMPLEv2, SUBJECTv2, CLINPATHv2, DATAv2 = update_tables_to_CDEv2(tables_path, CDEv1, CDEv2)


In [66]:

export_root = Path.cwd() / "clean/team-Hardy/v2"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)


In [67]:

STUDYv2.to_csv( export_root / "STUDY.csv")
PROTOCOLv2.to_csv(export_root / "PROTOCOL.csv")
SAMPLEv2.to_csv(export_root / "SAMPLE.csv")
SUBJECTv2.to_csv(export_root / "SUBJECT.csv")
CLINPATHv2.to_csv(export_root / "CLINPATH.csv")
DATAv2.to_csv(export_root / "DATA.csv")


Transfer cleaned metadata to raw buckets 




## transfer metadata Hardy 

In [68]:
# Hardy
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/hardy-credentials.json 


Activated service account credentials for: [raw-admin-hardy@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [69]:

!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-hardy/"


        20  2023-08-15T15:13:44Z  gs://asap-raw-data-team-hardy/test-raw-hardy.txt#1692112424343862  metageneration=1
        19  2023-08-17T09:58:09Z  gs://asap-raw-data-team-hardy/testfile.txt#1692266289818249  metageneration=1
                                 gs://asap-raw-data-team-hardy/fastqs/
                                 gs://asap-raw-data-team-hardy/metadata/
                                 gs://asap-raw-data-team-hardy/ogmetadata/
TOTAL: 2 objects, 39 bytes (39 B)


## fix bucket file structure to match the prescribed convention

change "fastq" -> "fastqs"

In [4]:

!gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-hardy/fastq" "gs://asap-raw-data-team-hardy/fastqs"


Copying gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_I1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_I1_001.fastq.gz...
Copying gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_I2_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_I2_001.fastq.gz...
Copying gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_R1_001.fastq.gz...
Copying gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_R2_001.fastq.gz [Content-Type=application/

In [6]:
!gsutil -u dnastack-asap-parkinsons ls "gs://asap-raw-data-team-hardy/fastqs" 

gs://asap-raw-data-team-hardy/fastqs/all_fastq.txt
gs://asap-raw-data-team-hardy/fastqs/220520_A01366_0202_AHL5JVDSX3/
gs://asap-raw-data-team-hardy/fastqs/220706_A01897_0009_AHMYFTDSX3/
gs://asap-raw-data-team-hardy/fastqs/220706_A01897_0010_BHMYM5DSX3/
gs://asap-raw-data-team-hardy/fastqs/220725_A01897_0013_BHLNTGDSX3/
gs://asap-raw-data-team-hardy/fastqs/220725_A01897_0014_AHN3MVDSX3/
gs://asap-raw-data-team-hardy/fastqs/220811_A01897_0015_BH3NFNDSX5/
gs://asap-raw-data-team-hardy/fastqs/220811_A01897_0016_AHF7KJDMXY/
gs://asap-raw-data-team-hardy/fastqs/221013_A01897_0034_AHGFTNDMXY/


In [70]:

# !gsutil -u dnastack-asap-parkinsons cp -r ./clean/team-Hardy/v2_20231128  "gs://asap-raw-data-team-hardy/metadata/v2"
!gsutil -u dnastack-asap-parkinsons cp -r "./clean/team-Hardy/v2_20231201/*.csv"  "gs://asap-raw-data-team-hardy/metadata/v2"


Copying file://./clean/team-Hardy/v2_20231201/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/SUBJECT.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/SAMPLE.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/DATA.csv [Content-Type=text/csv]...
- [4 files][552.2 KiB/552.2 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Hardy/v2_20231201/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/PROTOCOL.csv [Content-Type=text/csv]...
\ [6 files][556.5 KiB/556.5 KiB]                                                
Operation completed over 6 objects/556.5 KiB.                                   

### copy to workflow-dev bucket

First copy each set of metadata locally ...

In [72]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/wf-credentials.json

Activated service account credentials for: [admin-workflow-dev@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [73]:
!gsutil  cp -r "./clean/team-Hardy/v2_20231201/*.csv" "gs://asap-workflow-dev/metadata/v2/hardy"

Copying file://./clean/team-Hardy/v2_20231201/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/SUBJECT.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/SAMPLE.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/DATA.csv [Content-Type=text/csv]...
- [4 files][552.2 KiB/552.2 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Hardy/v2_20231201/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/PROTOCOL.csv [Content-Type=text/csv]...
\ [6 files][556.5 KiB/556.5 KiB]                                                
Operation completed over 6 objects/556.5 KiB.                                   

## check file md5s

In [35]:
from utils.checksums import extract_md5_from_details, extract_md5_from_details2


In [35]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/hardy-credentials.json  



Activated service account credentials for: [raw-admin-hardy@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [36]:

# !gcloud storage hash "gs://asap-raw-data-team-hardy/**/*.gz"  --skip-crc32c --hex  --billing-project dnastack-asap-parkinsons > hardy_hexhash.log

!gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-hardy/**/*.gz" > hardy_hexhash.log


using the module's C extension, so checksumming will run very slowly. For help
installing the extension, please see "gsutil help crcmod".



In [37]:
bucket_files_md5 = extract_md5_from_details2("hardy_hexhash.log")



checksum = DATAv2[['file_name','file_MD5']]
checksum['check1'] = checksum['file_MD5'].str.strip()
checksum['check2'] = checksum['file_name'].map(bucket_files_md5)




/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_56961/3763221414.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check1'] = checksum['file_MD5'].str.strip()
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_56961/3763221414.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check2'] = checksum['file_name'].map(bucket_files_md5)


In [1]:
! pip freeze

anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1693488585952/work
appnope @ file:///home/conda/feedstock_root/build_artifacts/appnope_1649077682618/work
argcomplete==1.12.3
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1692818318753/work
argon2-cffi-bindings @ file:///Users/runner/miniforge3/conda-bld/argon2-cffi-bindings_1695386808436/work
arrow @ file:///home/conda/feedstock_root/build_artifacts/arrow_1696128962909/work
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1698341106958/work
async-lru @ file:///home/conda/feedstock_root/build_artifacts/async-lru_1690563019058/work
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1683424013410/work
Babel @ file:///home/conda/feedstock_root/build_artifacts/babel_1698174530262/work
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
backports.functools-lru-cache @ file:///home/conda/feedstock_root/build_artifacts/backport

In [38]:
checksum[checksum.check1 != checksum.check2].file_name.to_list()


[]

In [46]:
checksum.head()

,file_name,file_MD5,check
0,babom_ACG_S20_L008_R1_001.fastq.gz,F37738d04879c1cb8c380920964dad4d,f37738d04879c1cb8c380920964dad4d
1,babom_ACG_S20_L008_I1_001.fastq.gz,F3053c59a33ce48cbea6a602dcc491c7,f3053c59a33ce48cbea6a602dcc491c7
2,babom_ACG_S20_L008_R2_001.fastq.gz,69fe97e968dc1cb782a3d995918b5bc8,69fe97e968dc1cb782a3d995918b5bc8
3,babom_ACG_S20_L008_I2_001.fastq.gz,4eeee300289a4fd18169dddbbfd8ad72,4eeee300289a4fd18169dddbbfd8ad72
4,babom_ACG_S20_L007_R1_001.fastq.gz,7b321fc0e55ffbd496a241dcb0746def,7b321fc0e55ffbd496a241dcb0746def


In [ ]:
# these are actually matches... probably unstripped line endings.